In [1]:
import numpy as np
from ler import LeR

In [2]:
ler = LeR(nsamples=10000)

Given: IMR waveform
psds not given. Choosing bilby's default psds
given psds:  {'L1': 'aLIGO_O4_high_asd.txt', 'H1': 'aLIGO_O4_high_asd.txt', 'V1': 'AdV_asd.txt'}
getting stored interpolator...
In case if you need regeneration of interpolator of the given gwsnr param, please delete this file, ./interpolator_pickle/halfSNR_dict_5.pickle


In [3]:
ler.gw_param = 'default'

In [4]:
len(ler.gw_param['zs'])

1000

In [5]:
unlensed = ler.unlensed_cbc_statistics(nsamples=1000)

chosen batch size = 25000. If you want to change batch size, self.batch_size = new_size
There will be 1 batche(s)
Batch no. 1
sampling gw source params...
calculating snrs...


In [4]:
unlensed = ler.unlensed_cbc_statistics(nsamples=100, resume=True)

chosen batch size = 25000. If you want to change batch size, self.batch_size = new_size
There will be 1 batche(s)
resuming from ./gw_params.json
no json file found with name ./gw_params.json. Set resume=False


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
ler.unlensed_rate()

getting gw_params from json file...
total unlensed rate with step function: 746.9979838263328
total unlensed rate with pdet function: 823.3578573267076


(746.9979838263328, 823.3578573267076)

In [7]:
lensed = ler.lensed_cbc_statistics(nsamples=10000)

chosen batch size = 25000. If you want to change batch size, self.batch_size = new_size
There will be 1 batche(s)
Batch no. 1
sampling lensed params...
sampling PEMD lens galaxy (with external shear) parameters...
solving lens equations...


100%|███████████████████████████████████████████████████████| 10000/10000 [00:08<00:00, 1247.22it/s]


calculating snrs...


In [8]:
rate  = ler.lensed_rate(lensed_param=lensed, none_as_nan=False)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
total lensed rate with step function: 0.7828224277502607
total lensed rate with pdet function: 0.7623564507410304


In [13]:
selecting_n_lensed_detectable_events

[0.7137498605958259, 0.7162724492595195]

In [15]:
ler.lensed_param_detectable = 'default'

In [19]:
test = ler.lensed_param_detectable

In [20]:
test = None

In [4]:
test = ler.selecting_n_lensed_detectable_events()

NameError: name 'os' is not defined

In [ ]:
from ler.helperroutines import append_json
import numpy as np

In [2]:
#dict_ = dict(a=np.array([7,8]),b=np.array([3,4]))
dict_ = dict(a=[9,10],b=[3,4])
fname = 'test.json'

In [3]:
append_json(fname, dict_)

In [4]:
not(True)

False

In [18]:
20/11

1.8181818181818181

In [24]:
30000//25000 +1

2

In [20]:
int(20/11)

1

In [25]:
5000//25000

0

In [5]:
cc=None
if cc:
    print('yes')
else:
    print('no')

no


In [23]:
25000//25000 +1

2